In [ ]:
import my_datalab_functions_example
from logging import Logger
from seeq import spy
from typing import Any
REQUEST: Any
LOG: Logger

In [ ]:
# This is a Data Lab Functions context cell since it is not annotated as a REST endpoint. Its contents will be executed only once, which is when the kernel is first executed.
from datetime import datetime
executed_timestmap = datetime.now()

In [ ]:
# GET /hello
#
# Visit https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/hello

{
    "message": f"Hello, from Data Lab!! This kernel was initially executed at {executed_timestmap}."
}

In [ ]:
# GET /version
#
# Visit https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/version

{
    "version": spy.server_version
}

In [ ]:
# GET /view-querystring
#
# Visit https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/view-querystring?id=5&name=Seeq

REQUEST['args']

In [ ]:
# POST /view-payload
#
# Using an API client such as Postman, send a POST request with the following body to https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/view-payload
# {"message": "This is my Data Lab Functions POST request!"}

REQUEST['body']

In [ ]:
# POST /combine
#
# Using an API client such as Postman, send a POST request with the following body to https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/combine
# {"idA": "<itemIdA>", "idB": "<itemIdB>", "op": "+", "workbookId": "<workbookId>", "worksheetId": "<worksheetId>"}
#
# Then view the logs by visting https://<my-seeq-server>/data-lab/<project-id>/functions/logs

import pandas as pd

# Get params from request body
idA = REQUEST['body']['idA']
idB = REQUEST['body']['idB']
op = REQUEST['body']['op']
workbook_id = REQUEST['body']['workbookId']
worksheet_id = REQUEST['body']['worksheetId']

# Create the formula to add the two signals
metadata = pd.DataFrame([{
    'Name': f'Combined Signal',
    'Formula': f"$signalA.setUnits('') {op} $signalB.setUnits('')",
    'Formula Parameters': {
        'signalA': idA,
        'signalB': idB
    },
    'Type': 'Formula'
}])

# Push the formula to Seeq
LOG.info(f"Pushing formula for 'Combined Signal' to workbook {workbook_id}")
combined_signal = spy.push(workbook=workbook_id, metadata=metadata)

# Get the current worksheet
workbook = spy.workbooks.pull(workbook_id)[0]
worksheet = next((ws for ws in workbook.worksheets if ws.id == worksheet_id), None)

# Add the combined signal if it's not already displayed on the worksheet
combined_signal_id = combined_signal['ID'].values[0]
if combined_signal_id not in worksheet.display_items['ID'].values:
    LOG.info(f"Adding signal {combined_signal_id} to worksheet {worksheet_id}")
    worksheet.display_items = pd.concat([worksheet.display_items, combined_signal]).reset_index()

# Push the updated worksheet back to Seeq
LOG.info(f"Updating worksheet {worksheet_id} in workbook {workbook_id}")
results = spy.workbooks.push(workbook)
results.to_json()

In [ ]:
# POST /module-example/combine
#
# To develop Data Lab Functions as a traditional Python module implementation, move your API endpoit logic to a Python module structure and invoke the desired module method while passing in the REQUEST object
#
# Using an API client such as Postman, send a POST request with the following body to https://<my-seeq-server>/data-lab/<project-id>/functions/notebooks/api/endpoints/module-example/combine
# {"idA": "<itemIdA>", "idB": "<itemIdB>", "op": "+", "workbookId": "<workbookId>", "worksheetId": "<worksheetId>"}
#
# Then view the logs by visting https://<my-seeq-server>/data-lab/<project-id>/functions/logs

my_datalab_functions_example.api.combine(REQUEST, LOG)